# Background
In recent year, vision transformer has been proposed and achieved state-of-the-art performance on image classification tasks. The key idea of vision transformer is to treat image as a sequence of patches and apply transformer to the sequence. However, the model also has its downside. For example, the model is computationally expensive and memory intensive. In this project, we want to introduce deformable vision transformer, which is a variant of vision transformer. It can improve Vision Transformer by only focusing on important regions of an image, instead of uniformly attending to all areas. Which could reduce the amount of input tokens for the self-attention layer.
We will first implement the vanilla [vision transformer](https://arxiv.org/abs/2010.11929) based on our current needle framework and then implement the [deformable vision transformer](https://arxiv.org/abs/2201.00520) and compare the performance of these two models on the CIFAR-10 dataset. We will also implement all the necessary components like operations or nn layers that are not implemented in the previous homeworks.

## Dataset
Before running the code, please download the CIFAR-10 dataset from [here](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz) and extract the files to the `data` folder. The below code copied from homework will load the dataset.

In [ ]:
import urllib.request
import os

!mkdir -p './data/ptb'
# Download Penn Treebank dataset
ptb_data = "https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb."
for f in ['train.txt', 'test.txt', 'valid.txt']:
    if not os.path.exists(os.path.join('./data/ptb', f)):
        urllib.request.urlretrieve(ptb_data + f, os.path.join('./data/ptb', f))

# Download CIFAR-10 dataset
if not os.path.isdir("./data/cifar-10-batches-py"):
    urllib.request.urlretrieve("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", "./data/cifar-10-python.tar.gz")
    !tar -xvzf './data/cifar-10-python.tar.gz' -C './data'

## Vision Transformer

The vanilla vision transformer is used as our baseline model. We followed the same architecture as the original paper. The model consists of a patch embedding layer, a positional encoding layer, and a transformer encoder. The patch embedding layer is used to convert the input image into a sequence of flattened patches. The positional embedding layer is used to add positional information to the patches. The transformer encoder is used to process the patches and generate the output. The model is trained using the cross-entropy loss function. We will get through each layer in the following sections.

### Patch Embedding Layer
The patch embedding layer takes the input (B, C, H, W) and returns the output (B, N, D), where B is the batch size, C is the number of channels, H is the height of the image, W is the width of the image, N is the number of patches, and D is the dimension of the patch embedding. It first splits the input into non-overlapping patches and then flattens each patch into a vector. A linear layer is used to map the flattened patches to the patch embedding.
Note that we first tried to use the convolutional layer to extract the patches, which is a more elegant way to implement. However, we met some shape issues when calculating the backward pass with even number of strides and even size of kernels. Therefore, we decided to implement this layer with multiple for loops with stack and split operations.

- **code**
The source code is in `python/needle/nn/nn_ViT.py`.

- **test code**
Please run the following code to test the patch embedding layer.

In [3]:
!python3 -m pytest ./tests/proj/test_ViT.py -l -v -k "patch_embedding" -s

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0 -- /root/anaconda3/envs/hw4_env/bin/python3
cachedir: .pytest_cache
rootdir: /CMU_10714_Work/CMU_10714_Deep_Learning_Systems/final_project/ViTNeedle
collecting ... Using needle backend
collected 8 items / 4 deselected / 4 selected                                  

tests/proj/test_ViT.py::test_patch_embedding[cpu-img_size0-16-3-768-4] PASSED
tests/proj/test_ViT.py::test_patch_embedding[cpu-img_size0-16-3-768-8] PASSED
tests/proj/test_ViT.py::test_patch_embedding[cpu-img_size1-4-3-64-4] PASSED
tests/proj/test_ViT.py::test_patch_embedding[cpu-img_size1-4-3-64-8] PASSED

======================= 4 passed, 4 deselected in 1.56s ========================


### Vision Transformer Block
We built the vision transformer block on top of the transformer encoder implemented in previous homeworks. We followed the structure proposed in the original vision transformer paper. The block consists of layer normalization, multi-head self-attention, and feed-forward layers, as illustrated in previous figure. The block takes the input (B, N, D) and returns the output (B, N, D), where B is the batch size, N is the number of patches plus a classification token, and D is the dimension of the patch embedding.

- **code**
The source code is in `python/needle/nn/nn_ViT.py`.

- **test code**
Please run the following code to test the vision transformer block.

In [4]:
!python3 -m pytest ./tests/proj/test_ViT.py -l -v -k "vision_transformer_block" -s

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0 -- /root/anaconda3/envs/hw4_env/bin/python3
cachedir: .pytest_cache
rootdir: /CMU_10714_Work/CMU_10714_Deep_Learning_Systems/final_project/ViTNeedle
collecting ... Using needle backend
collected 8 items / 6 deselected / 2 selected                                  

tests/proj/test_ViT.py::test_vision_transformer_block[cpu-0.0-65-16-1-16-64-2] PASSED
tests/proj/test_ViT.py::test_vision_transformer_block[cpu-0.0-65-16-1-16-64-4] PASSED

======================= 2 passed, 6 deselected in 0.60s ========================


### Vision Transformer
We built the vision transformer model with all the components implemented above. The model consists of a patch embedding layer, a positional encoding layer, and a stack of vision transformer blocks. We defined the positional embedding layer in this module, and we also implemented the classification token as used in the original paper. The attended classification token is finally fed into a classification decoder to output the final classification result. The model takes the input (B, C, H, W) and returns the output (B, NC), where B is the batch size, C is the number of channels, H is the height of the image, W is the width of the image, NC is the number of classes. The model is trained using the cross-entropy loss function.

- **code**
The source code is in `python/needle/nn/nn_ViT.py`.

- **test code**
Please run the following code to test the vision transformer.

In [5]:
!python3 -m pytest ./tests/proj/test_ViT.py -l -v -k "vision_transformer" -s

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0 -- /root/anaconda3/envs/hw4_env/bin/python3
cachedir: .pytest_cache
rootdir: /CMU_10714_Work/CMU_10714_Deep_Learning_Systems/final_project/ViTNeedle
collecting ... Using needle backend
collected 8 items / 4 deselected / 4 selected                                  

tests/proj/test_ViT.py::test_vision_transformer_block[cpu-0.0-65-16-1-16-64-2] PASSED
tests/proj/test_ViT.py::test_vision_transformer_block[cpu-0.0-65-16-1-16-64-4] PASSED
tests/proj/test_ViT.py::test_vision_transformer[cpu-0.0-6-img_size0-4-3-10-16-1-16-64-4] PASSED
tests/proj/test_ViT.py::test_vision_transformer[cpu-0.0-6-img_size0-4-3-10-16-1-16-64-8] PASSED

======================= 4 passed, 4 deselected in 2.42s ========================
